In [5]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from io import StringIO
from google.colab import files

# Membuat direktori untuk menyimpan file (opsional, untuk keperluan HTML)
output_dir = "grafik_perdagangan"
os.makedirs(output_dir, exist_ok=True)

# Data CSV dari permintaan sebelumnya
inventory_csv = """inventory_id,product_id,product_name,warehouse_id,warehouse_location,stock_date,stock_quantity,min_stock_level
INV001,P1,Laptop X1,WH1,Warehouse Jakarta,2025-02-01,400,100
INV002,P1,Laptop X1,WH2,Warehouse Singapore,2025-02-01,200,50
INV003,P2,T-Shirt Basic,WH3,Warehouse Bandung,2025-02-15,1200,200
INV004,P3,Choco Bar,WH1,Warehouse Jakarta,2025-03-05,1500,300
INV005,P2,T-Shirt Basic,WH3,Warehouse Bandung,2025-03-15,900,200"""

production_csv = """production_id,product_id,product_name,category,production_date,quantity_produced,production_cost,factory_location
PRD001,P1,Laptop X1,Elektronik,2025-01-10,500,450.5,Factory A
PRD002,P1,Laptop X1,Elektronik,2025-02-15,300,460.0,Factory B
PRD003,P2,T-Shirt Basic,Pakaian,2025-01-20,1000,5.2,Factory C
PRD004,P3,Choco Bar,Makanan,2025-03-01,2000,1.1,Factory A
PRD005,P2,T-Shirt Basic,Pakaian,2025-03-10,800,5.0,Factory C"""

sales_csv = """sale_id,product_id,product_name,sale_date,quantity_sold,sale_price,region,customer_type
SALE001,P1,Laptop X1,2025-02-10,100,600.00,Asia,Retail
SALE002,P1,Laptop X1,2025-02-20,50,580.00,Eropa,Wholesale
SALE003,P2,T-Shirt Basic,2025-02-25,500,10.00,Asia,Retail
SALE004,P3,Choco Bar,2025-03-10,800,2.00,Amerika,Retail
SALE005,P2,T-Shirt Basic,2025-03-15,300,9.50,Asia,Wholesale"""

# Menyimpan data ke file CSV
with open('inventory.csv', 'w') as f:
    f.write(inventory_csv)
with open('production.csv', 'w') as f:
    f.write(production_csv)
with open('sales.csv', 'w') as f:
    f.write(sales_csv)

# Memuat dataset
df_inventory = pd.read_csv('inventory.csv')
df_production = pd.read_csv('production.csv')
df_sales = pd.read_csv('sales.csv')

# 1. Jumlah Produksi vs Penjualan per Produk
produksi_per_produk = df_production.groupby('product_name')['quantity_produced'].sum().reset_index()
penjualan_per_produk = df_sales.groupby('product_name')['quantity_sold'].sum().reset_index()
prod_vs_jual = pd.merge(produksi_per_produk, penjualan_per_produk, on='product_name', how='outer').fillna(0)

# Visualisasi Bar Plot untuk Produksi vs Penjualan
fig1 = go.Figure(data=[
    go.Bar(name='Produksi', x=prod_vs_jual['product_name'], y=prod_vs_jual['quantity_produced'], marker_color='#1f77b4'),
    go.Bar(name='Penjualan', x=prod_vs_jual['product_name'], y=prod_vs_jual['quantity_sold'], marker_color='#ff7f0e')  # Diperbaiki: prod_s_jual -> prod_vs_jual
])
fig1.update_layout(
    title='Jumlah Produksi vs Penjualan per Produk',
    xaxis_title='Nama Produk',
    yaxis_title='Jumlah (Unit)',
    barmode='group',
    font=dict(size=12),
    margin=dict(l=50, r=50, t=100, b=50)
)
fig1.show()  # Menampilkan grafik interaktif

# 2. Total Penjualan per Produk
df_sales['total_revenue'] = df_sales['quantity_sold'] * df_sales['sale_price']
penjualan_total = df_sales.groupby('product_name')['total_revenue'].sum().reset_index()

# Visualisasi 3D Scatter Plot untuk Total Penjualan
fig2 = go.Figure(data=[
    go.Scatter3d(
        x=penjualan_total['product_name'],
        y=penjualan_total['total_revenue'],
        z=[0] * len(penjualan_total),  # Z-axis untuk efek 3D
        mode='markers+text',
        marker=dict(size=10, color=penjualan_total['total_revenue'], colorscale='Viridis', showscale=True),
        text=penjualan_total['product_name'],
        textposition='top center'
    )
])
fig2.update_layout(
    title='Total Penjualan per Produk',
    scene=dict(
        xaxis_title='Nama Produk',
        yaxis_title='Total Pendapatan (USD)',
        zaxis_title='Dummy Z-Axis'
    ),
    font=dict(size=12),
    margin=dict(l=50, r=50, t=100, b=50)
)
fig2.show()  # Menampilkan grafik interaktif

# 3. Total Stok Akhir per Produk
stok_akhir = df_inventory.groupby('product_name')['stock_quantity'].sum().reset_index()

# Visualisasi Bar Plot untuk Stok Akhir
fig3 = go.Figure(data=[
    go.Bar(name='Stok Akhir', x=stok_akhir['product_name'], y=stok_akhir['stock_quantity'], marker_color='#2ca02c')
])
fig3.update_layout(
    title='Total Stok Akhir per Produk',
    xaxis_title='Nama Produk',
    yaxis_title='Jumlah Stok (Unit)',
    font=dict(size=12),
    margin=dict(l=50, r=50, t=100, b=50)
)
fig3.show()  # Menampilkan grafik interaktif

# 4. Gabungan Produksi, Penjualan, dan Stok Akhir (3D Scatter Plot)
gabungan_data = pd.merge(prod_vs_jual, stok_akhir, on='product_name', how='outer').fillna(0)

# Visualisasi 3D Scatter Plot untuk Gabungan
fig4 = go.Figure(data=[
    go.Scatter3d(
        x=gabungan_data['product_name'],
        y=gabungan_data['quantity_produced'],
        z=gabungan_data['quantity_sold'],
        mode='markers',
        marker=dict(
            size=gabungan_data['stock_quantity'] / 50,  # Skala ukuran titik berdasarkan stok
            color=gabungan_data['stock_quantity'],
            colorscale='Plasma',
            showscale=True,
            colorbar_title='Stok Akhir (Unit)'
        ),
        text=gabungan_data['product_name'],
        textposition='top center'
    )
])
fig4.update_layout(
    title='Produksi, Penjualan, dan Stok Akhir per Produk',
    scene=dict(
        xaxis_title='Nama Produk',
        yaxis_title='Produksi (Unit)',
        zaxis_title='Penjualan (Unit)'
    ),
    font=dict(size=12),
    margin=dict(l=50, r=50, t=100, b=50)
)
fig4.show()  # Menampilkan grafik interaktif

# Menyimpan grafik sebagai HTML (opsional)
fig1.write_html(f"{output_dir}/produksi_vs_penjualan.html")
fig2.write_html(f"{output_dir}/total_penjualan.html")
fig3.write_html(f"{output_dir}/stok_akhir.html")
fig4.write_html(f"{output_dir}/gabungan.html")

# Menampilkan hasil analisis
print("Jumlah Produksi vs Penjualan per Produk:")
print(prod_vs_jual)
print("\nTotal Penjualan per Produk (USD):")
print(penjualan_total)
print("\nTotal Stok Akhir per Produk:")
print(stok_akhir)

# Mengunduh file HTML di Google Colab
print("\nMengunduh file HTML untuk grafik interaktif:")
for file in os.listdir(output_dir):
    files.download(f"{output_dir}/{file}")

# Petunjuk untuk menyimpan grafik secara manual
print("\nPetunjuk untuk menyimpan grafik:")
print("1. Grafik akan ditampilkan di notebook. Gunakan tombol kamera di toolbar grafik untuk mengunduh sebagai PNG.")
print("2. Alternatif: Ambil tangkapan layar grafik yang ditampilkan.")
print("3. File HTML interaktif telah disimpan di direktori 'grafik_perdagangan' dan diunduh otomatis.")
print("   Buka file HTML di browser untuk melihat grafik interaktif.")

Jumlah Produksi vs Penjualan per Produk:
    product_name  quantity_produced  quantity_sold
0      Choco Bar               2000            800
1      Laptop X1                800            150
2  T-Shirt Basic               1800            800

Total Penjualan per Produk (USD):
    product_name  total_revenue
0      Choco Bar         1600.0
1      Laptop X1        89000.0
2  T-Shirt Basic         7850.0

Total Stok Akhir per Produk:
    product_name  stock_quantity
0      Choco Bar            1500
1      Laptop X1             600
2  T-Shirt Basic            2100

Mengunduh file HTML untuk grafik interaktif:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Petunjuk untuk menyimpan grafik:
1. Grafik akan ditampilkan di notebook. Gunakan tombol kamera di toolbar grafik untuk mengunduh sebagai PNG.
2. Alternatif: Ambil tangkapan layar grafik yang ditampilkan.
3. File HTML interaktif telah disimpan di direktori 'grafik_perdagangan' dan diunduh otomatis.
   Buka file HTML di browser untuk melihat grafik interaktif.
